In [1]:
import psutil
import time
import threading

In [2]:
!pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 32.4 MB/s eta 0:00:00


In [3]:
import tenseal as ts
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
context.global_scale = 2**40

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt
plt.style.use("dark_background")

In [5]:
X_train=pd.read_csv('X_train3.csv')
X_train.head()

,Country,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population (in 100K),thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,6,2013,0,61.0,1,9.87,11734.853810,91.0,158,65.5,...,91.0,9.36,91.0,0.1,67792.338600,231.17,0.6,0.6,0.933,20.3
1,12,2008,1,181.0,0,7.22,51.252324,94.0,0,45.7,...,94.0,5.90,94.0,0.1,447.227960,0.36,3.5,3.4,0.699,12.8
2,54,2009,1,157.0,4,3.08,286.121145,97.0,0,45.9,...,97.0,8.84,97.0,0.5,1815.489496,0.84,2.3,2.2,0.603,11.3
3,52,2014,1,217.0,0,7.64,4.141293,98.0,0,45.9,...,97.0,5.25,98.0,0.3,43.823210,7.63,5.5,5.3,0.636,10.3
4,92,2014,1,119.0,1,6.74,1842.463782,8.0,0,57.1,...,8.0,8.30,8.0,0.1,12593.737400,3.94,1.9,1.8,0.780,13.0


In [6]:
y_train=pd.read_csv('y_train3.csv')
y_train.head()

,Life expectancy
0,82.5
1,69.6
2,73.4
3,66.0
4,77.6


In [7]:
X_test=pd.read_csv('X_test3.csv')
X_test.head()

,Country,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population (in 100K),thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,61,2005,1,161.0,1,3.59,16.975116,99.0,0,46.3,...,99.0,4.70,99.0,1.9,482.247620,27.45,1.9,1.7,0.706,12.4
1,35,2012,1,174.0,6,5.93,857.130704,74.0,0,55.8,...,85.0,4.26,85.0,0.4,5968.876767,1.15,3.3,3.2,0.706,13.1
2,35,2002,1,189.0,7,6.43,4.673592,63.0,0,45.1,...,74.0,5.79,82.0,2.4,38.465780,88.32,3.7,3.6,0.660,12.3
3,92,2009,1,127.0,1,6.87,1092.155356,85.0,0,53.4,...,85.0,7.56,85.0,0.1,7429.628274,35.79,2.0,1.9,0.755,12.8
4,8,2009,1,132.0,6,2.10,22.486384,74.0,0,47.3,...,84.0,5.85,81.0,0.1,495.294791,89.47,2.8,2.9,0.728,11.6


In [8]:
y_test=pd.read_csv('y_test3.csv')

In [9]:
X_train1=X_train.to_numpy()
X_train1[:5]

array([[6.00000000e+00, 2.01300000e+03, 0.00000000e+00, 6.10000000e+01,
        1.00000000e+00, 9.87000000e+00, 1.17348538e+04, 9.10000000e+01,
        1.58000000e+02, 6.55000000e+01, 1.00000000e+00, 9.10000000e+01,
        9.36000000e+00, 9.10000000e+01, 1.00000000e-01, 6.77923386e+04,
        2.31170000e+02, 6.00000000e-01, 6.00000000e-01, 9.33000000e-01,
        2.03000000e+01],
       [1.20000000e+01, 2.00800000e+03, 1.00000000e+00, 1.81000000e+02,
        0.00000000e+00, 7.22000000e+00, 5.12523242e+01, 9.40000000e+01,
        0.00000000e+00, 4.57000000e+01, 0.00000000e+00, 9.40000000e+01,
        5.90000000e+00, 9.40000000e+01, 1.00000000e-01, 4.47227960e+02,
        3.60000000e-01, 3.50000000e+00, 3.40000000e+00, 6.99000000e-01,
        1.28000000e+01],
       [5.40000000e+01, 2.00900000e+03, 1.00000000e+00, 1.57000000e+02,
        4.00000000e+00, 3.08000000e+00, 2.86121145e+02, 9.70000000e+01,
        0.00000000e+00, 4.59000000e+01, 5.00000000e+00, 9.70000000e+01,
        8.8400

In [10]:
y_train1=y_train.to_numpy()
y_train1[:5]

array([[82.5],
       [69.6],
       [73.4],
       [66. ],
       [77.6]])

In [11]:
X_test1=X_test.to_numpy()
X_test1[:5]

array([[6.10000000e+01, 2.00500000e+03, 1.00000000e+00, 1.61000000e+02,
        1.00000000e+00, 3.59000000e+00, 1.69751162e+01, 9.90000000e+01,
        0.00000000e+00, 4.63000000e+01, 1.00000000e+00, 9.90000000e+01,
        4.70000000e+00, 9.90000000e+01, 1.90000000e+00, 4.82247620e+02,
        2.74500000e+01, 1.90000000e+00, 1.70000000e+00, 7.06000000e-01,
        1.24000000e+01],
       [3.50000000e+01, 2.01200000e+03, 1.00000000e+00, 1.74000000e+02,
        6.00000000e+00, 5.93000000e+00, 8.57130704e+02, 7.40000000e+01,
        0.00000000e+00, 5.58000000e+01, 7.00000000e+00, 8.50000000e+01,
        4.26000000e+00, 8.50000000e+01, 4.00000000e-01, 5.96887677e+03,
        1.15000000e+00, 3.30000000e+00, 3.20000000e+00, 7.06000000e-01,
        1.31000000e+01],
       [3.50000000e+01, 2.00200000e+03, 1.00000000e+00, 1.89000000e+02,
        7.00000000e+00, 6.43000000e+00, 4.67359227e+00, 6.30000000e+01,
        0.00000000e+00, 4.51000000e+01, 8.00000000e+00, 7.40000000e+01,
        5.7900

In [12]:
X_train_augmented = np.hstack((np.ones((X_train1.shape[0], 1)), X_train1))
X_train_augmented

array([[1.000e+00, 6.000e+00, 2.013e+03, ..., 6.000e-01, 9.330e-01,
        2.030e+01],
       [1.000e+00, 1.200e+01, 2.008e+03, ..., 3.400e+00, 6.990e-01,
        1.280e+01],
       [1.000e+00, 5.400e+01, 2.009e+03, ..., 2.200e+00, 6.030e-01,
        1.130e+01],
       ...,
       [1.000e+00, 1.250e+02, 2.012e+03, ..., 2.400e+00, 7.390e-01,
        1.510e+01],
       [1.000e+00, 2.400e+01, 2.012e+03, ..., 5.000e-01, 9.070e-01,
        1.590e+01],
       [1.000e+00, 4.600e+01, 2.004e+03, ..., 1.100e+00, 8.770e-01,
        1.640e+01]])

In [13]:
X_train_T = X_train_augmented.T

In [14]:
X_train_T[:5]

array([[1.000e+00, 1.000e+00, 1.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [6.000e+00, 1.200e+01, 5.400e+01, ..., 1.250e+02, 2.400e+01,
        4.600e+01],
       [2.013e+03, 2.008e+03, 2.009e+03, ..., 2.012e+03, 2.012e+03,
        2.004e+03],
       [0.000e+00, 1.000e+00, 1.000e+00, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [6.100e+01, 1.810e+02, 1.570e+02, ..., 2.500e+01, 6.800e+01,
        8.600e+01]])

In [15]:
enc_X_train=[]
for i in X_train_T:
    p=ts.ckks_vector(context,i)
    enc_X_train.append(p)

In [16]:
enc_X_test=[]
for i in X_test1:
    p=ts.ckks_vector(context,i)
    enc_X_test.append(p)

In [17]:
y_train1=y_train1.reshape(-1)

In [18]:
enc_y_train=ts.ckks_vector(context,y_train1)

In [19]:
import os
from threading import Thread

In [20]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [21]:
pid = os.getpid()

# Duration to monitor CPU usage
monitor_duration = 45

In [22]:
time.sleep(15)

In [23]:
results={}
def train_lin_reg(enc_X_train,enc_y_train):
  time_start=time.time()
  enc_X_1=enc_X_train
  XTX=[]
  for i in enc_X_train:
      row_in=[]
      for j in enc_X_1:
          p=i.dot(j)
          row_in.append(p)
      XTX.append(row_in)

  XTY=[]
  for i in enc_X_train:
      p=i.dot(enc_y_train)
      XTY.append(p)

  XTX_dec=[]
  for i in XTX:
      row_in=[]
      for j in i:
          p=j.decrypt()
          row_in.append(p[0])
      XTX_dec.append(row_in)

  XTX_dec_1 = np.array(XTX_dec)

  XTY_dec=[]
  for i in XTY:
      p=i.decrypt()
      XTY_dec.append(p[0])

  XTY_dec_1 = np.array(XTY_dec)

  XTX_inv = np.linalg.inv(XTX_dec_1)
  theta = np.dot(XTX_inv, XTY_dec)

  results['bias'] = theta[0]
  results['weights'] = theta[1:]
  time_end=time.time()

  results['training time'] = time_end-time_start
  print(results['training time'])

In [24]:
train_thread = Thread(target=train_lin_reg, args=(enc_X_train, enc_y_train))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

CPU utilization: 97.2% | RAM utilization: 3.1872485978664398%
CPU utilization: 105.0% | RAM utilization: 3.2567721508510616%
CPU utilization: 92.3% | RAM utilization: 3.3521187378014003%
CPU utilization: 106.0% | RAM utilization: 3.4554108736642677%
CPU utilization: 85.9% | RAM utilization: 3.5606893967552664%
CPU utilization: 95.7% | RAM utilization: 3.6679543070743974%
CPU utilization: 106.0% | RAM utilization: 3.7851511535341893%
CPU utilization: 98.4% | RAM utilization: 3.832824447009359%
CPU utilization: 97.4% | RAM utilization: 3.8765249660282635%
CPU utilization: 113.9% | RAM utilization: 3.9559804551535462%
CPU utilization: 100.2% | RAM utilization: 4.055299816560149%
CPU utilization: 100.2% | RAM utilization: 4.1327689184573%
CPU utilization: 99.6% | RAM utilization: 4.224142730951374%
CPU utilization: 106.0% | RAM utilization: 4.307570994532921%
CPU utilization: 100.0% | RAM utilization: 4.424767840992712%
CPU utilization: 111.0% | RAM utilization: 4.536005525768108%
CPU util

In [25]:
results['weights']

array([-4.18871052e-04, -1.02222142e-01, -1.16752275e+00, -1.52015201e-02,
        1.20489729e-01, -1.10174031e-01,  1.54577643e-04, -6.15953295e-03,
       -2.27603653e-05,  4.40842627e-02, -8.90031688e-02,  6.58078890e-03,
       -2.20368979e-02,  1.44287370e-02, -4.47100155e-01,  4.60438444e-05,
        1.45213199e-04, -7.35346996e-02, -8.08579143e-02,  9.70057969e+00,
        8.21778563e-01])

In [26]:
results['bias']

260.83472686201276

In [27]:
results['training time']

34.76840329170227

In [28]:
weights=results['weights']
bias=results['bias']

predictions=[]

test_start=time.time()
for i in enc_X_test:
    t=i.dot(weights)
    t=t+bias
    predictions.append(t)

test_end=time.time()
results['testing time']=test_end-test_start
print(f"total time for testing = {test_end-test_start}")

total time for testing = 6.753183603286743


In [29]:
unenc_preds=[]
for i in predictions:
  unenc_preds.append(i.decrypt())

unenc_preds

[[71.22323914521239],
 [71.82026472153494],
 [69.58574604509346],
 [72.98250548172886],
 [71.62130624771231],
 [70.90505996854421],
 [51.123530153412375],
 [66.35384097649899],
 [69.82734258161511],
 [70.67607452186269],
 [73.01487027850052],
 [71.25704568381606],
 [64.65661656067147],
 [55.28899880795832],
 [84.461504994506],
 [51.44364660148095],
 [49.69487123271108],
 [70.50637333268578],
 [73.88581439012926],
 [66.57127137979201],
 [72.62943368280898],
 [70.6143573318698],
 [70.06932045528956],
 [71.03935841328163],
 [81.19730128953356],
 [58.57342519022842],
 [75.66916721117583],
 [51.74214851528028],
 [67.77612999095197],
 [82.23678352504665],
 [77.03518712233513],
 [72.65549567548162],
 [51.574200407653905],
 [75.67395484939736],
 [70.57475878612478],
 [68.43486142091966],
 [73.96710085592349],
 [68.35458854682874],
 [74.33857718397094],
 [73.68623021390256],
 [60.15776965666343],
 [52.5869356316568],
 [60.09715020250334],
 [67.66843234933928],
 [67.9273131561792],
 [72.00801349

In [30]:
!pip install joblib

In [31]:
import joblib
joblib.dump((weights, bias, unenc_preds, cpu_seconds, cpu_usages, ram_usages, results['training time'], results['testing time']), 'variables_ckks.pkl')

['variables_ckks.pkl']